# PatMatch Use With a Genome and Protein Sequences + Python

Here I'll demonstrate use of the stand-alone, command line based PatMatch on genomic nucleotide sequences and protein sequences. 
Hopefully, you'll see the the script you used previously to move to Python will work for such data cases as well.

### Preparing to use PatMatch software on genomic and protein sequence data



*Genome file*

As far as the genome, normally I'd recommnend obtaining the *Saccharomyces cerevisiae* budding yeast from [Illumina’s iGenomes site](https://support.illumina.com/sequencing/sequencing_software/igenome.html), and in fact, I have written out the commands necessary on the standard command line [here](http://fenglabwkshopmay2015.readthedocs.io/en/latest/genome%20of%20interest%20data%20acquisition/). [Illumina’s iGenomes site](https://support.illumina.com/sequencing/sequencing_software/igenome.html) is a collection of reference sequences and annotation files for commonly analyzed organisms.  However, it seems FTP-based and FTP sources won't work via Binder, and instead the genome file has been included already as `Sc_genome.fa`. (This has the advantage that it spares users from waiting a few seconds for the download to process.)

Alternatively, the following code pasted in a Jupyter notebook cell will work in Jupyter environments run via Binder to get the genome sequnce from [here](https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/). However, the genome file, `S288C_reference_sequence_R64-2-1_20150113.fsa`, from this source doesn't seem to contain nicely labeled chromosomes on the FASTA description lines and instead has cryptic alphanumerical code identifiers.

```
!curl -O https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
!tar -xzf S288C_reference_genome_Current_Release.tgz
!cp S288C_reference_genome_R64-2-1_20150113/S288C_reference_sequence_R64-2-1_20150113.fsa .
!rm -rf S288C_reference_genome_R64-2-1_20150113
!rm S288C_reference_genome_Current_Release.tgz
```

Given those cryptic labels, we'll stick with the included FASTA genome file.

*Protein sequences*

Following advice from [here](https://www.biostars.org/p/106065/#106110) is it is easy to select several and get quite a few at once from [here](http://www.uniprot.org/uniprot/?query=%28taxonomy%3A9606%29+AND+reviewed%3Ayes) because it formats a link when choose 'download' and designate 'uncompressed'. That link obtained can be used to make a `curl` command to get the text.  
Running the next cell will get a file of several human protein sequences.

In [ ]:
!curl "https://www.uniprot.org/uniprot/?sort=&desc=&compress=no&query=id:P62258%20OR%20id:P61981%20OR%20id:P31947%20OR%20id:P63104%20OR%20id:P30443%20OR%20id:Q16537%20OR%20id:Q30167%20OR%20id:Q95IE3%20OR%20id:Q5Y7A7%20OR%20id:P46952%20OR%20id:P31937%20OR%20id:P30542%20OR%20id:Q9NS82%20OR%20id:Q9NRG9%20OR%20id:Q2M2I8%20OR%20id:O43741%20OR%20id:P54619%20OR%20id:Q9UNQ0%20OR%20id:Q9H172%20OR%20id:Q9H222%20OR%20id:Q8N2K0%20OR%20id:Q96I13%20OR%20id:Q8IZP0%20OR%20id:P49748%20OR%20d:Q5HYA8%20OR%20id:Q9UHC1%20OR%20id:Q9UHV7%20OR%20id:P52732%20OR%20id:Q15058%20OR%20id:Q9P2E2%20OR%20id:Q9P2G9%20OR%20id:Q86Z14%20OR%20id:Q8NB78%20OR%20id:Q9Y4C1&format=fasta" > human_examples.txt

If that download fails, paste `!cp human_examples_bckup.txt human_examples.txt` into a cell and run it so that the later steps will still work.

### Running PatMatch on Genomic Sequences

You'll need the script you used prior to convert the results to a dataframe. To insure that is already here, you can run this next cell.

In [ ]:
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/patmatch-utilities/patmatch_results_to_df.py

### Running PatMatch on Protein Sequences

Prepare the FASTA file obtained per usual.

In [ ]:
!perl ../patmatch_1.2/unjustify_fasta.pl human_examples.txt

Now run `PatMatch`.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -p "STEP" human_examples.txt.prepared 1 s > human_protein.out

Bring the results into Python. Since we brought the script into where this notebook is running earlier, we can run the Python script already.

In [ ]:
%run patmatch_results_to_df.py human_protein.out --pattern STEP -name our_motif --protein_results

In [ ]:
import pandas as pd
df = pd.read_pickle("patmatch_pickled_df.pkl")
df

Cover somewhere

* Other ways to add the data to the running Binder are available using the file directory dashboard. EXPLAIN HOW TO UPLOAD using JUPYTER GUI.


* using `protein_results` flag

In [ ]:
# %load https://gist.githubusercontent.com/fomightez/b012e51ebef6ec58c1515df3ee0c850a/raw/300da6c67ceeaf5384a3e500648b993345c361cb/run_every_eight_mins.py
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()

<font color="#999">(Alternative ways to import data using the Jupyter environment's graphical user interface will be covered below.)</font>

Check the file listing by executing the next cell to see the FASTA-formatted file has been retrieved. Work through the following cells simialrly.

In [ ]:
!ls

Further following the PatMatch [USAGE](PatMatch initial demo and introduction.ipynb#Usage) information, sequences should be processed so that the lines of sequence data are formatted to one line for handling by PatMatch. The PatMatch authors have provided a utility script for doing that preparation step. The following cell will run that on the example data.

In [ ]:
!perl ../patmatch_1.2/unjustify_fasta.pl chrmt.fsa

That will produce a file with `.prepared` appended to the end of the supplied file name. 

Check that file was produced by running checking the file listing again using `!ls`.

In [ ]:
!ls

Having verified the prepared data file exists, you are ready to run the program to search for a pattern.

### Running PatMatch

The PatMatch [USAGE](PatMatch initial demo and introduction.ipynb#Usage) information says `-n` is for nucleotide pattern match and `-c` is for complementary strand; however, [based on my tests](PatMatch nucleic handling flags demystified.ipynb) it seems that `-c` means it is for the complementary strand **in addition to** the strand in the dataset. 

**<font color="red">Therefore, if you want the pattern search to be performed on BOTH strands of the supplied sequence, as is the default of the web-based PatMatch tools, you actually want to use the `-c` flag when authoring the command.</font>**

If you are curious about this aspect futher, I demonstrate that [here](PatMatch nucleic handling flags demystified.ipynb) and in the course of that cover how to replicate the three options typically offered for strand at PatMatch web-based offerings. Feel free to examine and run that notebook or simply use the `-c` flag if you are trying to scan both strands. 


In [ ]:
# !perl ../patmatch_1.2/patmatch.pl -n "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared #dataset strand only
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared

In [ ]:
!ls

That is the basics of running PatMatch. There are options you can add to control this mismatch amount and whether to allow insertions,deletiions, or substitutions towards thos mismtaches. Example:

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 1 ids

See the [USAGE](PatMatch initial demo and introduction.ipynb#Usage) for more information about those options. However, that covers the basics.

With the basics in hand, and using the power of the command line, searches of more sequences or more sequences and more patterns become possible. However, you'll quickly encounter problems handling all those results. As a simple example, we'll use the example pattern matching search we developed above as example for integrating with Python for more efficient handling of the results and to touch upon the advantanges offered by combining with a scripting language.

## Importing PatMatch Results into a Pandas Dataframe and Exporting to Excel

Now that you see what PatMatch is returning as results, you'll probably note that why that looks easy to read for a human, it isn't very computer friendly. Indeed, if you have used the web-based PatMatch offerings, you'll note that they return the results in a table form that is more useful.

Other ways to add the data to the running Binder are available using the file directory dashboard. EXPLAIN HOW TO UPLOAD using JUPYTER GUI.

In [ ]:
# from https://stackoverflow.com/a/42703609/8508004
import io
#import pandas as pd
output = !perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 1 ids
#df = pd.read_table(io.StringIO(output.n))
print(type(output)) # see http://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.text.html#IPython.utils.text.SList
print (output.n)

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared > test.out

In [ ]:
#!/usr/bin/env python
# patmatch_results_to_df.py
__author__ = "Wayne Decatur" #fomightez on GitHub
__license__ = "MIT"
__version__ = "0.1.0"


# patmatch_results_to_df.py by Wayne Decatur
# ver 0.1
#
#*******************************************************************************
# Verified compatible with both Python 2.7 and Python 3.6; written initially in 
# Python 3. (See below.)
#
#
# PURPOSE: Takes output from command line-based PatMatch and brings it into 
# Python as a dataframe and saves a file of that dataframe for use elsewhere. 
# Optionally, it can also return that dataframe for use inside a Jupyter 
# notebook.
#
# This script is meant to be a utility script for working with command 
# line-based PatMatch and Python, see a demonstration of use in
# https://github.com/fomightez/patmatch-binder/blob/master/notebooks/PatMatch%20with%20Python.ipynb
# 
# Assumes for nucleic acid patterns, it was run with `-c` flag and tries to 
# assign strand information.
# 
# See https://github.com/fomightez/patmatch-binder about PatMatch.
#
# Written to run from command line or pasted/loaded inside a Jupyter notebook 
# cell. 
#
#
#
#
#
#
# Verified compatible with both Python 2.7 and Python 3.6; written initially in 
# Python 3. 
#
#
# Dependencies beyond the mostly standard libraries/modules:
#
#
#
# VERSION HISTORY:
# v.0.1. basic working version

#
# To do:
# - verify compatible with 2.7 (use output from patmatch-binder) OTHERWISE, FIX TWO NOTES ABOVE ABOUT 2.7
# - add way to bring in pattern searched
# - add in way to signal nucleic or protein because with protein data won't need
#   strand handling
# - incorporate in demo notebook in patmatch-binder; also(?) in that demo binder 
#   show how to bring into Python theweb-based PatMatch data from the xls file?
#
#
#
#
# TO RUN:
# Examples,
# Enter on the command line of your terminal, the line
#-----------------------------------
# python patmatch_results_to_df.py -RESULTS_FILE
#-----------------------------------
# Issue `patmatch_results_to_df.py -h` for details.
# 
#
# When using in a notebook, if you don't specify dataframe objects, , you must
# instead supply strings of file names for the pickled dataframes in the call
# to the main function. 
# To use this after pasting or loading into a cell in a Jupyter notebook, in
# the next cell specify the two dataframes then call the main function similar 
# to below:
# pattern= "DDWDWTAWAAGTARTADDDD"
# df = patmatch_results_to_df("test.out", pattern=pattern)
# df
#
#
#
#
# 
#
'''
CURRENT ACTUAL CODE FOR RUNNING/TESTING IN A NOTEBOOK WHEN LOADED OR PASTED IN 
ANOTHER CELL:
pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df("test.out", pattern=pattern)
df
'''
#
#
#*******************************************************************************
#





#*******************************************************************************
##################################
#  USER ADJUSTABLE VALUES        #

##################################
#

## Settings and options for output plot 
df_save_as_name = 'patmatch_pickled_df.pkl' # name for saving pickled dataframe

#
#*******************************************************************************
#**********************END USER ADJUSTABLE VARIABLES****************************


















#*******************************************************************************
#*******************************************************************************
###DO NOT EDIT BELOW HERE - ENTER VALUES ABOVE###

import sys
import os
import pandas as pd




#*******************************************************************************
###------------------------'main' function of script---------------------------##

def patmatch_results_to_df(
    results_file, pattern="?", return_df = True, pickle_df=True):
    '''
    Main function of script. 
    It will take a file of results from command line-based PatMatch and make
    a dataframe that will be more useful with Python/othergenetic-oriented 
    scripts.
    Optionally also returns a dataframe of the results data. Meant for use in 
    a Jupyter notebook.
    '''
    # Bring in the necessary data:
    #---------------------------------------------------------------------------

    with open(results_file, 'r') as the_file:
        results = the_file.read()

    # feedback
    sys.stderr.write("Provided results read...")


    # Parse:
    #---------------------------------------------------------------------------
    results = results.split('>')
    # remove blanks
    results = [x for x in results if x]

    # prepare to give some unique indentifiers to each match
    identifiers=[]
    if pattern == "?":
        id_prefix = "pattern-"
    elif len(pattern) > 29:
        id_prefix = pattern[:25] + "...-"
    else:
        id_prefix = pattern + "-"

    matching_patterns = []
    starts = []
    ends = []
    strand_info = []
    for indx,each in enumerate(results):
        each_part = each.split()
        first_line, matching_pattern = each_part[0].strip(),each_part[1].strip()
        # Because I wanted to include `.strip()`, it seemed I couldn't do last
        # two lines all as `first_line, matching_pattern= each.split()[:2]`

        # Parse numbers between the brackets in first line(first, I will split
        # on the ':' just in case the extracted first line has `[]` in first 
        # part)
        second_half = first_line.split(":")[1]
        nums_str = second_half[second_half.find("[")+1:second_half.find("]")]
        nums = nums_str.split(",")[:2]
        start,end = nums[0],nums[1]
        # Determine strand. 
        # I noticed when `-c` flag is used the first number in the interval 
        # returned to indicate the location will be larger than the second for 
        # those on the negative strand. 
        if start > end:
            strand = -1
            # fix start and end so start is actually lowest value to be 
            # consistent with system I have been using of late (like Ensembl)
            start, end = end,start

        else:
            strand = 1
        assert start < end ,"The 'start' value should be lower; strand \
        information is handled by `strand` property."
        identifiers.append(id_prefix+str(indx+1)) #`+1` so numbering more 
        # tpyical than python zero-indexing
        matching_patterns.append(matching_pattern)
        starts.append(start)
        ends.append(end)
        strand_info.append(strand)

    # Make collected results into dataframe and improve on it
    #---------------------------------------------------------------------------
    df = pd.DataFrame(list(zip(
        identifiers, starts, ends,strand_info, matching_patterns)),
        columns=['seq_id', 'start','end','strand','matching pattern'])
    # add query pattern as a column
    df['query pattern'] = pattern

    # better re-order the columns(?)


    #print(updated_sites_df)#originally for debugging during development,added..
    # Document the full set of data collected in the terminal or 
    # Jupyter notebook display in some manner. 
    # Using `df.to_string()` because more universal than `print(df)` 
    # or Jupyter's `display(df)`.
    sys.stderr.write( "\nFor documenting purposes, the following lists the "
        "parsed data:\n")
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #    display(df)
    sys.stderr.write(df.to_string())

    # Handle pickling the modified sites dataframe
    if pickle_df == False:
        sys.stderr.write("\n\nA dataframe of the parsed data shown above "
        "was not stored for use\nelsewhere "
        "because `no_pickling` was specified in place of the output file name.")
    else:
        df.to_pickle(df_save_as_name )
        # Let user know
        sys.stderr.write( "\n\nA dataframe of the parsed data shown above "
        "has been\nsaved as a file in a manner where other "
        "Python programs\ncan access it (pickled form).\n"
        "RESULTING DATAFRAME is stored as ==> '{}'".format(df_save_as_name ))

    # optionally, return df
    if return_df:
        sys.stderr.write( "\n\nReturning a dataframe with the information "
                "as well.")
        return df

###--------------------------END OF MAIN FUNCTION----------------------------###
###--------------------------END OF MAIN FUNCTION----------------------------###












#*******************************************************************************
###------------------------'main' section of script---------------------------##

def main():
    """ Main entry point of the script """
    # placing actual main action in a 'helper'script so can call that easily 
    # with a distinguishing name in Jupyter notebooks, where `main()` may get
    # assigned multiple times depending how many scripts imported/pasted in.
    kwargs = {}
    if args.pattern:
        kwargs['pattern'] = args.pattern
    else:
        kwargs['pattern'] = '?'
    if df_save_as_name == 'no_pickling':
        kwargs['pickle_df'] = False
    kwargs['return_df'] = False #probably don't want dataframe returned if 
    # calling script from command line
    patmatch_results_to_df(results_file,**kwargs)
    # using https://www.saltycrane.com/blog/2008/01/how-to-use-args-and-kwargs-in-python/#calling-a-function
    # to build keyword arguments to pass to the function above
    # (see https://stackoverflow.com/a/28986876/8508004 and
    # https://stackoverflow.com/a/1496355/8508004 
    # (maybe https://stackoverflow.com/a/7437238/8508004 might help too) for 
    # related help)





if __name__ == "__main__" and '__file__' in globals():
    """ This is executed when run from the command line """
    # Code with just `if __name__ == "__main__":` alone will be run if pasted
    # into a notebook. The addition of ` and '__file__' in globals()` is based
    # on https://stackoverflow.com/a/22923872/8508004
    # See also https://stackoverflow.com/a/22424821/8508004 for an option to 
    # provide arguments when prototyping a full script in the notebook.
    ###-----------------for parsing command line arguments-----------------------###
    import argparse
    parser = argparse.ArgumentParser(prog='patmatch_results_to_df.py',
        description="patmatch_results_to_df.py \
        Takes output from command line-based PatMatch and brings it into \
        Python as a dataframe and saves a file of that dataframe for use \
        elsewhere. Optionally, it can also return that dataframe for use \
        inside a Jupyter notebook. Meant to be a utility script for working \
        with command line-based PatMatch and Python.\
        Assumes for nucleic acid patterns, it was run with `-c` flag and tries \
        to assign strand. \
        **** Script by Wayne Decatur   \
        (fomightez @ github) ***")

    parser.add_argument("results_file", help="Name of file of PatMatch results \
        file to parse.\
        ", metavar="RESULTS_FILE")
    parser.add_argument('-patt', '--pattern', action='store', type=str, 
        help="**OPTIONAL** Pattern used to perform the pattern matching search \
        that generated the results. The resulting dataframe will more \
        informative if one is provided; however, it is not essential. To \
        provide the pattern simply enter the text after the flag. For example, \
        if the search had been for an EcoRI site, include `--pattern GAATTC`, \
        without quotes or ticks, in the call to the script.")
    parser.add_argument("-p", "--protein_results",help=
    "add this flag to indicate the data are from a pattern match of protein \
    sequences. Otherwise it assumed the results are from pattern matching on \
    nucleic acid sequences.", action="store_true")
    parser.add_argument('-dfo', '--df_output', action='store', type=str, 
    default= df_save_as_name, help="OPTIONAL: Set file name for saving pickled \
    dataframe. If none provided, '{}' will be used. To force no dataframe to \
    be saved, enter `-dfo no_pickling` without quotes as output file \
    (ATYPICAL).".format(df_save_as_name))




    #I would also like trigger help to display if no arguments provided because 
    # need at least one for url
    if len(sys.argv)==1:    #from http://stackoverflow.com/questions/4042452/display-help-message-with-python-argparse-when-script-is-called-without-any-argu
        parser.print_help()
        sys.exit(1)
    args = parser.parse_args()
    results_file= args.results_file
    df_save_as_name = args.df_output


    main()

#*******************************************************************************
###-***********************END MAIN PORTION OF SCRIPT***********************-###
#*******************************************************************************




In [ ]:
pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df("test.out", pattern=pattern)
df

In [ ]:
# %load https://gist.githubusercontent.com/fomightez/b012e51ebef6ec58c1515df3ee0c850a/raw/300da6c67ceeaf5384a3e500648b993345c361cb/run_every_eight_mins.py
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()